In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/cassava-leaf-disease-classification'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## import all necessary libraries required for this project
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os 
import torchvision
import albumentations as A
import os
from pathlib import Path
from tqdm import tqdm
import plotly.graph_objects as go
import plotly_express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.optimizers import Adam
import warnings
warnings.simplefilter("ignore")
import os, cv2, json
from PIL import Image
from tensorflow.keras.layers import Flatten,LSTM, Dense, Flatten, Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.models import Sequential 
from keras_preprocessing.text import Tokenizer 
from keras.initializers import glorot_uniform 
from sklearn import model_selection
 

In [ ]:
#### locate the files required for analysis
WORK_DIR = '../input/cassava-leaf-disease-classification'
os.listdir(WORK_DIR)
print('Train images: %d' %len(os.listdir(
    os.path.join(WORK_DIR, "train_images"))))

with open(os.path.join(WORK_DIR, "label_num_to_disease_map.json")) as file:
    print(json.dumps(json.loads(file.read()), indent=4))
    
train_labels = pd.read_csv(os.path.join(WORK_DIR, "train.csv"))
train_labels.head()



In [ ]:
#### Define the constants/hyperparameter
BATCH_SIZE = 8
STEPS_PER_EPOCH = len(train_labels)*0.8 / BATCH_SIZE
VALIDATION_STEPS = len(train_labels)*0.2 / BATCH_SIZE
EPOCHS = 20
TARGET_SIZE = 350

In [ ]:
### Define paths required.
base_path = Path('../input/cassava-leaf-disease-classification')
train_img_dir = base_path /'train_images'
test_img_dir = base_path /'test_images'

In [ ]:
### Define the dataframes
train_df = pd.read_csv(base_path/'train.csv')
diseaseMapping = pd.read_json(base_path/'label_num_to_disease_map.json', typ='series')

train_images = os.listdir(base_path/'train_images/')
test_images = os.listdir(base_path/'test_images/')

In [ ]:
diseaseMapping

In [ ]:
mappingDict = diseaseMapping.to_dict()

In [ ]:
### Start understanding of data
train_df.head()

In [ ]:
train_df = train_df.replace(mappingDict)

In [ ]:
## Visualisation of each disease
labelCounts = train_df['label'].value_counts().reset_index()
labelCounts.columns = ['Label', 'Number of Observations']

fig = px.pie(labelCounts, names = 'Label',values='Number of Observations', labels = mappingDict, color_discrete_sequence=px.colors.sequential.YlOrBr)


fig.show()


In [ ]:
uniqueIds = train_df['image_id'].nunique()
if(uniqueIds == len(train_df)):
    print('There are no repeating Image IDs in the dataset')
else:
    print(f'There are {len(train_df) - uniqueIds} repeating Image IDs')

In [ ]:
print(len(train_images))

In [ ]:
healthyImages = train_df[train_df['label'] == 'Healthy']['image_id'].to_list()
cbbImages = train_df[train_df['label'] == 'Cassava Bacterial Blight (CBB)']['image_id'].to_list()
cbsdImages = train_df[train_df['label'] == 'Cassava Brown Streak Disease (CBSD)']['image_id'].to_list()
cgmImages = train_df[train_df['label'] == 'Cassava Green Mottle (CGM)']['image_id'].to_list()
cmdImages = train_df[train_df['label'] == 'Cassava Mosaic Disease (CMD)']['image_id'].to_list()

In [ ]:
def showImages(images):

    # Extract 9 random images from it
    random_images = [np.random.choice(images) for i in range(9)]

    # Adjust the size of your images
    plt.figure(figsize=(10,8))

    # Iterate and plot random images
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        img = plt.imread(train_img_dir/random_images[i])
        plt.imshow(img, cmap='gray')
        plt.axis('off')

    # Adjust subplot parameters to give specified padding
    plt.tight_layout()   

In [ ]:
def showHistogram(sample_img, title):
    f = plt.figure(figsize=(16,8))
    f.add_subplot(1,2, 1)

    raw_image = plt.imread(train_img_dir/sample_img)
    plt.imshow(raw_image, cmap='gray')
    plt.colorbar()
    plt.title(title)
    print(f"Image dimensions:  {raw_image.shape[0],raw_image.shape[1]}")
    print(f"Maximum pixel value : {raw_image.max():.1f} ; Minimum pixel value:{raw_image.min():.1f}")
    print(f"Mean value of the pixels : {raw_image.mean():.1f} ; Standard deviation : {raw_image.std():.1f}")

    f.add_subplot(1,2, 2)

    #_ = plt.hist(raw_image.ravel(),bins = 256, color = 'orange',)
    _ = plt.hist(raw_image[:, :, 0].ravel(), bins = 256, color = 'red', alpha = 0.5)
    _ = plt.hist(raw_image[:, :, 1].ravel(), bins = 256, color = 'Green', alpha = 0.5)
    _ = plt.hist(raw_image[:, :, 2].ravel(), bins = 256, color = 'Blue', alpha = 0.5)
    _ = plt.xlabel('Intensity Value')
    _ = plt.ylabel('Count')
    _ = plt.legend(['Red_Channel', 'Green_Channel', 'Blue_Channel'])
    plt.show()


In [ ]:
def load_image(image_id):
    image = cv2.imread(str(train_img_dir/image_id))
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def showChannelDistribution(images, leafType):
    imageArray = [load_image(image_id) for image_id in images]
    image
    red_values = [np.mean(imageArray[idx][:, :, 0]) for idx in range(len(imageArray))]
    green_values = [np.mean(imageArray[idx][:, :, 1]) for idx in range(len(imageArray))]
    blue_values = [np.mean(imageArray[idx][:, :, 2]) for idx in range(len(imageArray))]
    values = [np.mean(imageArray[idx]) for idx in range(len(imageArray))]
    
    hist_data = [red_values, green_values, blue_values, values]
    group_labels = ['Red', 'Green', 'Blue', 'All']

    fig = ff.create_distplot(hist_data, group_labels,colors = ['red', 'green','blue','grey'])
    fig.update_layout(template = 'plotly_white', title_text = f'Channel Distribution - {leafType}')
    fig.show()
    return hist_data

In [ ]:
def showBoxPlot(histData, leafType):
    figData = []
    for i, name in zip(range(3), ['Red', 'Green', 'Blue']):
        trace = go.Box(y = histData[i], name = name, boxpoints='all', marker_color  = name)
        figData.append(trace)

    fig = go.Figure(figData)
    fig.update_layout(title_text = f'Pixel Intensity Distribution - {leafType}', template = 'plotly_white')
    fig.show() 

In [ ]:
showImages(healthyImages)

In [ ]:
showHistogram(healthyImages[0], 'Healthy Image')

In [ ]:
data = showChannelDistribution(healthyImages, 'Healthy')

In [ ]:
showBoxPlot(data, 'Healthy Leaves')

In [ ]:
showImages(cbbImages)

In [ ]:
showHistogram(cbbImages[0], 'CBB Image')

In [ ]:
data = showChannelDistribution(cbbImages, 'CBB Images')

In [ ]:
showBoxPlot(data, 'CBB Images')

In [ ]:
showImages(cbsdImages)

In [ ]:
showHistogram(cbsdImages[0], 'CBSD Image')

In [ ]:
data = showChannelDistribution(cbsdImages, 'CBSD Images')

In [ ]:
showImages(cgmImages)

In [ ]:
showHistogram(cgmImages[0], 'CGM Image')

In [ ]:
data = showChannelDistribution(cgmImages, 'CGM Images')

In [ ]:
showBoxPlot(data, 'CGM Images')

In [ ]:
showImages(cmdImages)

In [ ]:
showHistogram(cmdImages[0], 'CMD Image')

In [ ]:
data = showChannelDistribution(cmdImages[:2000], 'CMD Images')

In [ ]:
showBoxPlot(data, 'CMD Images')

In [ ]:
channelIntensityDf = pd.DataFrame(
    {
        'Leaf Type' : ['Healthy', 'CBB','CBSD', 'CGM', 'CMD'], 
        'Red Channel Mean' : [108,102,106,113,110],
        'Green Channel Mean' : [126,117,123,128,128],
        'Blue Channel Mean' : [80,66,72,85,80]
    }
)

channelIntensityDf.style.background_gradient(cmap='Greens', axis = 0)

In [ ]:
train_labels.label = train_labels.label.astype('str')

In [ ]:
train_datagen = ImageDataGenerator(validation_split = 0.2,
                                     preprocessing_function = None,
                                     rotation_range = 45,
                                     zoom_range = 0.2,
                                     horizontal_flip = True,
                                     vertical_flip = True,
                                     fill_mode = 'nearest',
                                     shear_range = 0.1,
                                     height_shift_range = 0.1,
                                     width_shift_range = 0.1)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(train_labels,
                         directory = os.path.join(WORK_DIR, "train_images"),
                         subset = "training",
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         batch_size = BATCH_SIZE,
                         class_mode = "sparse")

In [ ]:
validation_datagen = ImageDataGenerator(validation_split = 0.2)

In [ ]:
validation_generator = validation_datagen.flow_from_dataframe(train_labels,
                         directory = os.path.join(WORK_DIR, "train_images"),
                         subset = "validation",
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         batch_size = BATCH_SIZE,
                         class_mode = "sparse")


In [ ]:
def create_model():
    conv_base = EfficientNetB3(include_top = False, weights = None,
                               input_shape = (TARGET_SIZE, TARGET_SIZE, 3))
    model = conv_base.output
    model = layers.GlobalAveragePooling2D()(model)
    model = layers.Dense(5, activation = "softmax")(model)
    model = models.Model(conv_base.input, model)

    model.compile(optimizer = Adam(lr = 0.001),
                  loss = "sparse_categorical_crossentropy",
                  metrics = ["acc"])
    return model

In [ ]:
model = create_model()
model.summary()

In [ ]:
model.save('./EffNetB0_512_8.h5')

In [ ]:
print('Our EfficientNet CNN has %d layers' %len(model.layers))

In [ ]:
ss = pd.read_csv(os.path.join(WORK_DIR, "sample_submission.csv"))
ss

In [ ]:
preds = []


In [ ]:
for image_id in ss.image_id:
    image = Image.open(os.path.join(WORK_DIR,  "test_images", image_id))
    image = image.resize((TARGET_SIZE, TARGET_SIZE))
    image = np.expand_dims(image, axis = 0)
    preds.append(np.argmax(model.predict(image)))
 

In [ ]:
y_predict = model.predict(image)
y_predict

In [ ]:
#preds = np.round(preds,4)

print("prediction is :", preds)

#my_submission = pd.DataFrame(preds, columns=['image_id', 'label'])
#submission.label = preds
#my_submission.to_csv('submission.csv', index=False)
#submission.to_csv('submission.csv', index=False)

#image_names = [img_name.numpy().decode('utf-8') for img, img_name in iter(test_images.unbatch())]



In [ ]:
df = {'image_id' : image_id, 'label' : preds}
#print(df)
df

In [ ]:
#my_submission = pd.DataFrame(df)
#print(my_submission)
#my_submission

In [ ]:
#ss = pd.DataFrame(df)
#ss

In [ ]:
from keras.models import load_model
prev_model = load_model('EffNetB0_512_8.h5')

In [ ]:
top_model = Sequential()
top_model.add(Embedding(TARGET_SIZE, 20, input_length=TARGET_SIZE)) 
top_model.add(LSTM(32,dropout=0.2, recurrent_dropout=0.2)) 
top_model.add(Dense(1, activation='sigmoid'))


In [ ]:
from keras import Model
model_new = Model(inputs=prev_model.input, outputs=top_model(prev_model.output))

In [ ]:
model_new.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc']) 
print(model_new.summary())

In [ ]:
pred_new =[]
for image_id in ss.image_id:
    image = Image.open(os.path.join(WORK_DIR,  "test_images", image_id))
    image = image.resize((TARGET_SIZE, TARGET_SIZE))
    image = np.expand_dims(image, axis = 0)
    pred_new.append(np.argmax(model_new.predict(image)))

In [ ]:
print("prediction is :", pred_new)

In [ ]:
y_predict_new = model_new.predict(image)
y_predict_new

In [ ]:
#my_submission.to_csv('submission.csv', index=False)

In [ ]:
ss["label"] = pred_new
ss.to_csv('submission.csv', index=False)

